# CSUMB Diagnostics Lab Tool Development

YOU and you're team are bioinformaticians for a clinical lab that performs diagnostic testing for the early, and accurate, detection of differentially expressed oncogenes and known oncogenic mutations.

Today, you're tasked with developing some software that can automate the quantification of gene expression and the detection of variants from RNA-sequencing data.

Case-study data has been deposited in the organizational database: `drive/Shareddrives/CSUMB-GREAT` where you can access it to help you develop these tools.

### First things first:

**MOUNT DRIVE IN LEFT PANEL!!**

We need to navigate to the directory that stores all our data

The `%` line is used by google colab to execute "magic" functions -- in this case, we just want to re-locate to the CSUMB-GREAT directory in `Shareddrives`

In [22]:
# for me, since I own the drive, I use this path
%cd drive/MyDrive/CSUMB-GREAT
# for you, since its shared, use:
# %cd drive/Shareddrives/CSUMB-GREAT

[Errno 2] No such file or directory: 'drive/MyDrive/CSUMB-GREAT'
/content/drive/MyDrive/CSUMB-GREAT


`%pwd` will confirm our location

In [ ]:
%pwd

## Loading reference sequences (FASTA) into Python

In [16]:
# this DICTIONARY will hold
# genes as KEYS
# sequences as VALUES
reference_dictionary = {}

# with the file connection set to our reference FASTA ... 
with open("data/reference/three_gene_ref.fa") as reference:
        header = '' # create empty string for headers
        sequence = '' # create empty string for sequences

        for line in reference:
            if line.startswith('>'):
                header = line[1:].rstrip()
                sequence = ''
            else:
                sequence += ''.join(line.rstrip().split()).upper()
            reference_dictionary[header] = sequence

In [17]:
for gene, sequence in reference_dictionary.items():
  print(gene, len(sequence)/1000, 'kb')

CDH1 4.811 kb
KRAS 5.43 kb
TP53 2.512 kb


## Loading Alignments (SAM) into Python

The `.sam` alignment files are stored in `data/aligned_data/{KRAS, CDH1, TP53}`

In [18]:
%ls data/aligned_data/*/pair_1 | head

data/aligned_data/CDH1/pair_1:
pair_1.sam

data/aligned_data/KRAS/pair_1:
pair_1.sam

data/aligned_data/TP53/pair_1:
pair_1.sam


We can also use some `bash` commands by pre-pending them with `!`, for example let's look at one of the alignment (.sam) files and think about which lines we need to load and which we need to skip

In [19]:
!head data/aligned_data/KRAS/pair_3/pair_3.sam

@HD	VN:1.5	SO:unsorted	GO:query
@SQ	SN:CDH1	LN:4811
@SQ	SN:KRAS	LN:5430
@SQ	SN:TP53	LN:2512
@PG	ID:bowtie2	PN:bowtie2	VN:2.5.1	CL:"./bowtie2-2.5.1-linux-x86_64/bowtie2-align-s -x bowtie_index/ -1 input_data/KRAS/pair_3/g12c_50_R1.fastq.header_modified_R1.fastq -2 input_data/KRAS/pair_3/g12c_50_R2.fastq.header_modified_R2.fastq -S aligned_data/KRAS/pair_3/pair_3.sam --no-unal"
511677	65	KRAS	130	42	75M	=	2026	1971	AAGGCGGCGGCGGGGCCAGAGGCTCAGCGGCTCCCAGGTGCGGGAGAGAGGCCTGCTGAAAATGACTGAATATAA	@GID@CD>><DF<FH=@@I>BHAB?<H=@>HAEEH=>DB>GD?EGG<@BH==EBFD<ECEB@DIC><??IDGEGF	AS:i:0	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:75	YS:i:0	YT:Z:DP
511677	129	KRAS	2026	42	75M	=	130	-1971	ACTCTGTGGTGGTCCTGCTGACAAATCAAGAGCATTGCTTTTGTTTCTTAAGAAAACAAACTCTTTTTTAAAAAT	@GID@CD>><DF<FH=@@I>BHAB?<H=@>HAEEH=>DB>GD?EGG<@BH==EBFD<ECEB@DIC><??IDGEGF	AS:i:0	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:75	YS:i:0	YT:Z:DP
540285	65	KRAS	2139	42	75M	=	3208	1144	TGGTGGTGTGCCAAGACATTAATTTTTTTTTTAAACAATGAAGTGAAAAAGTTTTACAATCTCTAGGTTT

### And now,

Lets write the code.

I'm providing the destination object: `alignment_dictionary` that you will populate with `read_id`'s, `read_start` positions, and `read_sequence`'s

Use the `FASTA` parser code above as inspiration to implement an approach that reads the `SAM` files line-by-line and stores the relevant information in `alignment_dictionary`

In [ ]:
alignment_dictionary = {}
# keys: read_id
# values: tuple(start position of alignment, sequence of aligned read)

# define an object to hold the name of the SAM file you want to load
pre_path = 'data/aligned_data' # this part of the path is constant; all .sams are here

gene = 'KRAS' # gene is defined as you need it
#gene = 'TP53'
#gene = 'CDH1'

pair = 'pair_3' # pair is defined as you need it

# then, we stitch all of these strings together with `+` and creat a full path
input_sam = pre_path + '/' + gene + '/' + pair + '/' + pair + ".sam" # pick one .sam for now...
# ex. data/aligned_data/KRAS/pair_3/pair_3.sam

print('loading ' + input_sam + ' into aligment dictionary')

# start here, test and try things, populate the dictionary
with open(input_sam) as alignment:
        for line in alignment:
          print(line)


Explore `alignment_dictionary` by printing its keys and values

# Exercises

## Exercise #1

Calculate the number of reads aligned to the reference and the size of your reference gene
  - Determine how many reads we have aligned to our reference if possible, compare to the bowtie output and remember that in this special case, we're aligning to only one gene
  
hint:
  - we want to measure how many entries are in our alignment obj...

## Exercise #2

Calculate the number of reads aligned to the reference per-kilobase length of gene
  - Scale / normalize the read abundance to the size of the gene and answer: **why are we doing this**?
  

hint:
  - reuse `read_count`

## Exercise #3

Calculate per-nucleotide coverage across target gene
  - for every position in the reference, determine the number of reads that overlap that position

hint:
  - Python is 0-based (refer to primer) but the reference sequence is not
  - with that said, you have reads and their start positions **on the reference**, you need to convert the read position to the reference position using this information ..... 

extra credit:
  - using `max(dictionary, key=dictionary.get)` determine the position, and coverage that is the maximum (do the same for `min`) across the reference

## Exercise #4

Calculate SNP-aware per-nucleotide coverage of target reference
  - for every position in the reference,determine the number of reads that overlap that position with the expected, matching nucleotide as well as the number of reads that contain single-nucleotide polymorphisms (SNPs)

hint:
  - re-use as much code as possible....

extra credit:
  - calculate the "allele frequency" at positions with
  - mutations, where `AF = reference observed / total observations`

## (Advanced) Exercise #5:

Calculate SNP-aware per-nucleotide coverage of target reference
using the `MD:Z` tag in the SAM file, use this to calculate
Allele Frequency
  - you must create this *without* using the previous objects created to enable the other variant calling approach

### Step 1

Parse your `.sam` file of choice for the MD:Z tag column

hint:
  - start with the code you deployed for SAM parsing [above](https://colab.research.google.com/drive/1Cp-BX7JXC-n_BDiVP3IB1RhVIhTjf084#scrollTo=G0YXdc89JUfg&line=13&uniqifier=1)

In [21]:
md_dictionary = {}
# keys: read_id
# values: tuple(start position of alignment, MD tag)

### Step 2

Implement the scanning and calculation

hint:
  - you need the same information as you did before...